# Scraping Google Trends

Quick tryout to scrap the top 20 Google Trends in France and the US. The goal of this file is to be excuted everyday at 23h59 to get a fair glimpse of the state of local internet.

In [57]:
from bs4 import BeautifulSoup
import requests
import urllib.request
import sys
import os
import traceback
import re
import time
import json
import unidecode


We're gonna try with one headers first. If this doesn't work, we're gonna try proxy rotation and multi-headers to hide our identity. But given the fact that we are not going to scrap a lot of data, we might be able to go anonymous.

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:62.0) Gecko/20100101 Firefox/62.0'}

Our main URL will be Google Trends' RSS flux (easier), we'll then try for each country.

In [39]:
url = 'https://trends.google.fr/trends/trendingsearches/daily/rss?geo=US'

Get the html file.

In [40]:
html = requests.get(url, headers=headers)

Create a Beautiful Soup object.

In [41]:
soup = BeautifulSoup(html.text, "lxml")

Get the list of all items available.

In [42]:
items = soup.findAll('item')
print("Number of items available :", len(countries))

Number of items available : 20


Get the most popular search in France.

In [43]:
search = soup.find('item')
print(search.title.text)

Champions League


Iterate over all 20 trends searchs, get all titles.

In [44]:
top_searchs = []
for item in items:
    top_searchs.append(item.title.text)
print(top_searchs)

['Champions League', 'Post Malone', 'Ricin', 'Real Madrid', 'Toys R Us', 'Amazon jobs', 'Venom review', 'Hailey Baldwin', 'CSKA Moscow vs Real Madrid', 'Christian Yelich', 'Fred Trump', 'Tom Petty', 'Juventus', 'LaMelo Ball', 'Ruby Rose', 'Jake Paul', 'Brooks Koepka', 'A Star Is Born Original', 'Anna Faris', 'Megyn Kelly']


Now get the top search for all countries with roman alpahabet.

In [60]:
countries_tag = ['US', 'FR', 'BE', 'ZA','DE', 'AR', 'AU', 'AT', 'BR', 'CA',
                 'CL', 'CO', 'CZ', 'DK', 'DE', 'HU', 'IE', 'IT', 'KE', 'MX',
                 'NL', 'NZ', 'PL', 'RO', 'GB', 'CH']
results = []
for tag in countries_tag:
    top_searchs = []
    url_country = 'https://trends.google.fr/trends/trendingsearches/daily/rss?geo=' + tag
    html_country = requests.get(url_country, headers=headers)
    soup = BeautifulSoup(html_country.text, "lxml")
    items = soup.findAll('item')
    for item in items:
        search = unidecode.unidecode(item.title.text)
        top_searchs.append(search)
    results.append(top_searchs)

dico_results = dict(zip(countries_tag, results))
print(dico_results)
with open("trends.json", "w") as fout:
    json.dump(dico_results, fout)

{'PL': ['Getback', 'Olga Boladz', 'Real Cska', 'Wiatr', 'Liga Mistrzow', 'Bayern - Ajax Amsterdam', 'Alert RCB', 'Juventus - Young Boys', 'Sapkowski', 'Unia Hrubieszow - Gornik Zabrze', 'Jozef Unrug', 'Sebastian Fabijanski', 'Grzegorz Russak', 'Liga Mistrzow 2018', 'Michal Szpak', 'Dziennikarz motoryzacyjny', 'Kasia Tusk', '1983', 'Protest sluzb mundurowych', 'Protest Policji'], 'AR': ['Rocio Guirao Diaz', 'Dia del Odontologo', 'Patricia Pacheco', 'CSKA Moscu contra Real Madrid', 'DEPORTES', 'Boca', 'Juventus', 'Sabat', 'Pollo Alvarez', 'Fernanda Meneses', 'Sol Perez', 'Barcelona', 'Anama Ferreira', 'ShowMatch', 'Del Potro', 'Luis Ortega', 'Estadio Monumental Antonio Vespucio Liberti', 'Estudiantes', 'Nicole Neumann', 'Manchester City'], 'CO': ['Millonarios', 'River Plate', 'Juventus', 'Bayern contra Ajax', 'Leo Dan', 'Manchester United', 'Genesis Rua', 'Manchester City', 'La Ley del Corazon 2 Rcn', 'Espn en Vivo', 'Juanes', 'Canal 1', 'Real Madrid', 'Enrique Colavizza', 'RCN', 'Charle

There we go, we got the top 20 Google Searches for 26 countries. Now watch the trends.sh script to set up this Python file to run every day at a given time.